In [ ]:
import tensorflow as tf


def formula(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Функція tf.function приймає у якості аргумента звичайну функцію 
# і генерує обчислювальний граф:
function_that_uses_graph = tf.function(formula)

# Ми будемо виконувати матричні операції, 
# тому перш ніж передавати аргументи на цю функцію потрібно перетворити їх на тензори:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = formula(x1, y1, b1).numpy()

# Після виконання обчислень функція function_that_uses_a_graph поверне тензор,
# який буде перетворено на багатовимірний масив numpy:
tf_function_value = function_that_uses_graph(x1, y1, b1).numpy()

assert(orig_value == tf_function_value)

x1.shape
y1.shape
b1.shape

TensorShape([])

При цьому в tensorflow можна зручно ***конвертувати numpy масив у тензор*** та назад. 

Наприклад, для того, щоб отримати тензор маючи багатовимірний масив numpy достатньо виконати наступний код:

In [ ]:
import numpy as np

x_array = np.array([1, 2, 3])
x_tensor = tf.convert_to_tensor(x_array)

При цьому перетворення ***тензора на багатовимірний масив numpy*** виглядає ще простіше:

In [7]:
x_tensor.numpy()

array([1, 2, 3])

##### `tf.function` можна використовувати як декоратор:

In [8]:
import tensorflow as tf


@tf.function
def function_that_uses_a_graph(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x


x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = formula(x1, y1, b1).numpy()
tf_function_value = function_that_uses_a_graph(x1, y1, b1).numpy()

assert(orig_value == tf_function_value)


У попередній tensorflow версії 1.x для того, щоб створити та виконати обчислювальний граф:

In [ ]:
## Old version doesn't work here
# import tensorflow as tf

# x, y = 2, 3

# add_op = tf.add(x, y, name='Add')
# mul_op = tf.multiply(x, y, name='Multiply')
# pow_op = tf.pow(add_op, mul_op, name='Power')
# useless_op = tf.multiply(x, add_op, name='Useless')

# with tf.Session() as sess:
#     pow_out, useless_out = sess.run([pow_op, useless_op])

У нейронних мережах часто використовується активаційна функція ReLU:

$$
f(x)=max(x,0)
$$

Ось як її можна було б реалізувати з використанням низькорівневого API tensorflow:

In [10]:
import tensorflow as tf

@tf.function
def relu_activation(x):
    if tf.greater(x, 0):
        return x
    return 0

print(relu_activation(tf.constant(1)).numpy())
print(relu_activation(tf.constant(-1)).numpy())

1
0


У машинному навчанні досить часто доводиться мати справу з обчисленням градієнтів. Це завдання виникає настільки часто, що на сьогоднішній день вже існує безліч бібліотек, які дозволяють виконувати автоматичне диференціювання. Такий механізм є і в tensorflow. 

Нехай є якась функція і нам потрібно обчислити значення її похідної у точці x0=2. Для цього спершу оголосимо функцію:

In [1]:
import tensorflow as tf


def f(x):
    return 1 / x ** 2


x = tf.Variable(2.0)
with tf.GradientTape() as tape:
    y = f(x)
    dydx = tape.gradient(y, x)
    print(dydx)

tf.Tensor(-0.25, shape=(), dtype=float32)


За допомогою класу `GradientTape` tensorflow записує всі операції, необхідні для обчислення похідної в змінну `tape`. Для того, щоб обчислити значення похідної в конкретній точці, потрібно викликати метод `gradient`, приймає 2 аргументи. Перший - функція, похідну якої ми хочемо обчислити. Другий - аргумент, за яким обчислюється похідна (причому це може бути тензор). Цей механізм нам ще знадобиться далі.